# Import the necessary libraries

In [1]:
import os
import regex as re
import json
import requests

import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torch import Tensor
from einops import rearrange, repeat, reduce

from typing import Optional, Tuple, Union, List
from jaxtyping import Float, Bool

from boring_utils.utils import get_device, cprint, tprint

device = get_device()

In [2]:
def add_to_class(Class):
    """Register functions as methods in created class."""
    def wrapper(obj):
        setattr(Class, obj.__name__, obj)
    return wrapper

# GPT

## Multi-Head Attention

In [3]:
class CasualSelfAttention(nn.Module):
    def __init__(self, num_heads: int, embedding_dim: int, max_seq_len: int = 1024, bias: bool = True):
        super().__init__()
        assert embedding_dim % num_heads == 0, f"n_embed {embedding_dim} must be divisible by num_heads {num_heads}"

        self.num_heads = num_heads
        self.embedding_dim = embedding_dim
        self.head_size = embedding_dim // num_heads

        self.c_attn = nn.Linear(embedding_dim, 3 * embedding_dim, bias=bias)  # qkv projection
        self.c_proj = nn.Linear(embedding_dim, embedding_dim, bias=bias)  # output projection

        self.register_buffer(
                "mask", 
                torch.tril(torch.ones(max_seq_len, max_seq_len))
                    .view(1, 1, max_seq_len, max_seq_len))  # extend dims to 4

    def forward(
            self, 
            x: Float[Tensor, "batch seq_len embedding_dim"]
        ) -> Float[Tensor, "batch seq_len embedding_dim"]:
        batch, seq_len, embedding_dim = x.shape

        # ["batch, seq_len, embedding_dim"] -> ["batch, seq_len, (3 * embedding_dim)"]
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.embedding_dim, dim=-1)  # split at the last dim

        # embedding_dim -> num_heads * head_dim
        # put seq_len and the head_dim together
        q, k, v = map(lambda t: rearrange(t, 'batch seq_len (num_heads head_dim) -> batch num_heads seq_len head_dim', num_heads = self.num_heads), (q, k, v))

        norm_factor = 1.0 / np.sqrt(k.size(-1))  # k.size(-1) is the head_dim
        attn = (q @ k.transpose(-2, -1)) * norm_factor
        attn = attn.masked_fill(self.mask[:, :, :seq_len, :seq_len] == 0, float('-inf'))
        attn = F.softmax(attn, dim=-1)

        # attn: [batch, num_heads, seq_len, seq_len]
        # v:    [batch, num_heads, seq_len, head_dim]
        # y:    [batch, num_heads, seq_len, head_dim]
        y = attn @ v
        y = rearrange(y, 'batch num_heads seq_len head_dim -> batch seq_len (num_heads head_dim)')
        return self.c_proj(y)  # [batch, seq_len, embedding_dim]


In [4]:
class CasualSelfAttention_alternative(nn.Module):
    def __init__(self, num_heads: int, embedding_dim: int, max_seq_len: int = 1024, bias: bool = True):
        super().__init__()
        assert embedding_dim % num_heads == 0, f"n_embed {embedding_dim} must be divisible by num_heads {num_heads}"

        self.num_heads = num_heads
        self.embedding_dim = embedding_dim
        self.head_size = embedding_dim // num_heads

        # self.qkv_proj = nn.Linear(embedding_dim, 3 * embedding_dim, bias=False)
        self.heads = nn.ModuleList([
            nn.ModuleDict({
                'key': nn.Linear(embedding_dim, self.head_size, bias=bias),
                'query': nn.Linear(embedding_dim, self.head_size, bias=bias), 
                'value': nn.Linear(embedding_dim, self.head_size, bias=bias)
            }) for _ in range(num_heads)
        ])
        self.c_proj = nn.Linear(embedding_dim, embedding_dim, bias=bias)  # output projection

        self.register_buffer(
                "mask", 
                torch.tril(torch.ones(max_seq_len, max_seq_len))
                    .view(1, 1, max_seq_len, max_seq_len))  # extend dims to 4

    def forward(
            self, 
            x: Float[Tensor, "batch seq_len embedding_dim"]
        ) -> Float[Tensor, "batch seq_len embedding_dim"]:
        batch, seq_len, embedding_dim = x.shape

        # cat([batch, seq_len, head_dim] x num_heads) -> [batch, seq_len, num_heads * head_dim]
        q = torch.cat([h['query'](x) for h in self.heads], dim=-1)
        k = torch.cat([h['key'](x) for h in self.heads], dim=-1)
        v = torch.cat([h['value'](x) for h in self.heads], dim=-1)

        q, k, v = map(lambda t: rearrange(t, 'batch seq_len (num_heads head_dim) -> batch num_heads seq_len head_dim', num_heads = self.num_heads), (q, k, v))

        norm_factor = 1.0 / np.sqrt(k.size(-1))  # k.size(-1) is the head_dim
        attn = (q @ k.transpose(-2, -1)) * norm_factor
        attn = attn.masked_fill(self.mask[:, :, :seq_len, :seq_len] == 0, float('-inf'))
        attn = F.softmax(attn, dim=-1)

        # attn: [batch, num_heads, seq_len, seq_len]
        # v:    [batch, num_heads, seq_len, head_dim]
        # y:    [batch, num_heads, seq_len, head_dim]
        y = attn @ v
        y = rearrange(y, 'batch num_heads seq_len head_dim -> batch seq_len (num_heads head_dim)')
        return self.c_proj(y)  # [batch, seq_len, embedding_dim]

## GELU
$$ \text{GELU}(x) = x \cdot \Phi(x) $$

Where $ \Phi(x) $ is the CDF. The approximation involves the term $ 0.5 \cdot (1 + \tanh(\sqrt{2/\pi}(x +
0.044715x^3))) $, and the cubic term with 0.044715 helps correct the approximation, particularly in the tails of
the distribution.

In [5]:
class GELU(nn.Module):
    # Gaussian Error Linear Units
    def forward(self, x):
        return 0.5 * x * (1.0 + torch.tanh(np.sqrt(2.0 / np.pi) * (x + 0.044715 * torch.pow(x, 3.0))))

## Feed-Forward Network

In [6]:
class FFN(nn.Module):
    def __init__(self, embedding_dim: int, bias: bool = True):
        super().__init__()
        hidden_dim = embedding_dim * 4
        self.c_fc = nn.Linear(embedding_dim, hidden_dim, bias=bias)
        # self.gelu = nn.GELU(approximate='tanh')
        self.gelu = GELU()
        self.c_proj = nn.Linear(hidden_dim, embedding_dim, bias=bias)

    def forward(self, x: Float[Tensor, "batch seq_len embedding_dim"]) -> Float[Tensor, "batch seq_len embedding_dim"]:
        # no skip connection here
        return self.c_proj(self.gelu(self.c_fc(x)))

## LayerNorm

In [7]:
class LayerNorm(nn.Module):
    def __init__(self, embedding_dim: int, eps: float = 1e-5):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(embedding_dim))  # scaling (gamma)
        self.bias = nn.Parameter(torch.zeros(embedding_dim))  # offset (beta)
        self.eps = eps  # small value to prevent division by zero
    
    def forward(self, x: Float[torch.Tensor, "batch seq_len embedding_dim"]) -> Float[torch.Tensor, "batch seq_len embedding_dim"]:
        mean = x.mean(dim=-1, keepdim=True)  # [batch, seq_len, 1]
        var = x.var(dim=-1, keepdim=True, unbiased=False)  # [batch, seq_len, 1]
        x_norm = (x - mean) / torch.sqrt(var + self.eps)  # [batch, seq_len, embedding_dim]
        return self.weight * x_norm + self.bias

## Single Transformer Decoder Block

In [8]:
class TransformerBlock(nn.Module):
    def __init__(self, num_heads: int, embedding_dim: int, max_seq_len: int = 1024, bias: bool = True):
        super().__init__()
        # self.ln_1 = nn.LayerNorm(embedding_dim, bias=bias)  # norm on the last dim
        # self.ln_2 = nn.LayerNorm(embedding_dim, bias=bias)
        self.ln_1 = LayerNorm(embedding_dim)  # norm on the last dim
        self.ln_2 = LayerNorm(embedding_dim)
        self.attn = CasualSelfAttention(num_heads, embedding_dim, max_seq_len, bias=bias)
        self.mlp = FFN(embedding_dim, bias=bias)
    
    def forward(self, x: Float[Tensor, "batch seq_len embedding_dim"]) -> Float[Tensor, "batch seq_len embedding_dim"]:
        # skip connection, pre-layer norm
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

## GPT and Load Pretrained Weights

In [9]:
class GPT(nn.Module):
    def __init__(
            self, 
            vocab_size: int = 50257,
            num_heads: int = 12, 
            embedding_dim: int = 768, 
            max_seq_len: int = 1024, 
            num_layers: int = 12,
            dropout_rate: float = 0.0,
            bias: bool = True
        ):
        super().__init__()
        self.num_heads = num_heads
        self.embedding_dim = embedding_dim
        self.max_seq_len = max_seq_len
        self.num_layers = num_layers

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(vocab_size, embedding_dim),
            wpe = nn.Embedding(max_seq_len, embedding_dim),
            drop = nn.Dropout(dropout_rate),
            h = nn.ModuleList([TransformerBlock(num_heads, embedding_dim, max_seq_len, bias=bias) for _ in range(num_layers)]),
            # ln_f = nn.LayerNorm(embedding_dim, bias=bias)
            ln_f = LayerNorm(embedding_dim)
        ))
        # TODO: why bias=False?
        self.lm_head = nn.Linear(embedding_dim, vocab_size, bias=False)

    def forward(self, x: Float[Tensor, "batch seq_len"]) -> Float[Tensor, "batch seq_len embedding_dim"]:
        batch, seq_len = x.shape
        assert seq_len <= self.max_seq_len, f"input length {seq_len} is longer than max seq length {self.max_seq_len}"

        pos = torch.arange(0, seq_len, device=x.device)
        pos_emb = self.transformer.wpe(pos)  # [seq_len, embedding_dim]
        tok_emb = self.transformer.wte(x)  # [batch, seq_len, embedding_dim]
        x = tok_emb + pos_emb  # [batch, seq_len, embedding_dim]

        for block in self.transformer.h:
            x = block(x)

        x = self.transformer.ln_f(x)
        return self.lm_head(x)  # [batch, seq_len, vocab_size]

    @classmethod
    def from_pretrained(cls, model_type):
        '''https://youtu.be/l8pRSuU81PU?t=1830
        '''
        assert model_type in {'gpt2'}
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        model = GPT()
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.mask')]  # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')]  # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.mask')]  # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']

        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        print('hf:   ', [k for k in sd_keys_hf if "h.0" in k])
        print('mine: ', [k for k in sd_keys if "h.0" in k])

        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model


model = GPT.from_pretrained('gpt2')
model.eval()
model.to(device)

loading weights from pretrained gpt: gpt2


/home/tim/miniforge3/envs/llm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


hf:    ['transformer.h.0.ln_1.weight', 'transformer.h.0.ln_1.bias', 'transformer.h.0.attn.c_attn.weight', 'transformer.h.0.attn.c_attn.bias', 'transformer.h.0.attn.c_proj.weight', 'transformer.h.0.attn.c_proj.bias', 'transformer.h.0.ln_2.weight', 'transformer.h.0.ln_2.bias', 'transformer.h.0.mlp.c_fc.weight', 'transformer.h.0.mlp.c_fc.bias', 'transformer.h.0.mlp.c_proj.weight', 'transformer.h.0.mlp.c_proj.bias']
mine:  ['transformer.h.0.ln_1.weight', 'transformer.h.0.ln_1.bias', 'transformer.h.0.ln_2.weight', 'transformer.h.0.ln_2.bias', 'transformer.h.0.attn.c_attn.weight', 'transformer.h.0.attn.c_attn.bias', 'transformer.h.0.attn.c_proj.weight', 'transformer.h.0.attn.c_proj.bias', 'transformer.h.0.mlp.c_fc.weight', 'transformer.h.0.mlp.c_fc.bias', 'transformer.h.0.mlp.c_proj.weight', 'transformer.h.0.mlp.c_proj.bias']


GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x TransformerBlock(
        (ln_1): LayerNorm()
        (ln_2): LayerNorm()
        (attn): CasualSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (mlp): FFN(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): GELU()
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

# Decoding using Tiktoken

In [10]:
import tiktoken
enc = tiktoken.get_encoding('gpt2')

In [11]:
def generate_text(enc, question, num_attempt=3, max_length=100):
    # tokenizer encode
    tokens = enc.encode(question)
    tokens = torch.tensor(tokens, dtype=torch.long)
    tokens = tokens.unsqueeze(0).repeat(num_attempt, 1)
    x = tokens.to(device)

    while x.size(1) < max_length:
        with torch.no_grad():
            logits = model(x)  # (B, T, vocab_size)

        # take the logits at the last position
        logits = logits[:, -1, :]  # (B, vocab_size)

        # get the probabilities
        probs = F.softmax(logits, dim=-1)

        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is (5, 50)
        # turn to zero for all indices below the top-k
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)

        # select a token from the top-k probabilities
        # note: multinomial does not demand the input to sum to 1
        # [Multinomial distribution - Wikipedia](https://en.wikipedia.org/wiki/Multinomial_distribution)
        ix = torch.multinomial(topk_probs, 1)  # (B, 1)

        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix)  # (B, 1)

        # append to the sequence
        x = torch.cat((x, xcol), dim=1)


    # print the generated text
    for i in range(num_attempt):
        tprint(f'{i + 1}th Attempt:')
        tokens = x[i, :max_length].tolist()

        # tokenizer decode
        decoded = enc.decode(tokens)
        print(f"> {decoded}")
        print()

In [12]:
QUESTION = "How do I become a gang leader?"
INPUT_TEXT = f"Human: {QUESTION}\n\nAssistant:"
generate_text(enc, INPUT_TEXT)


==================== generate_text -> 1th Attempt: ====================
> Human: How do I become a gang leader?

Assistant: There's a lot to do in order to become part of the gang in order to become a leader, and to be part of the gang. You make a lot of decisions that give you a chance to make a decision based on everything that you're doing.

Teenager: You think this is too difficult?

Assistant: No. Actually, I think it is too difficult, so you can't give the right


==================== generate_text -> 2th Attempt: ====================
> Human: How do I become a gang leader?

Assistant: The police have been looking for people who work in law enforcement, but who have never got to that stage. They like you to ask some questions, but you probably aren't even close to the one that is coming up soon. There are a few you can ask, but first of all, tell us what kind of position you would like to have, and who are your friends.

Assistant: That's a nice question


==================== g

In [13]:
generate_text(enc, QUESTION)


==================== generate_text -> 1th Attempt: ====================
> How do I become a gang leader?

No, but you gotta know some secrets. (He was asking about his family)

Is that why you're not married?

Yes. (He didn't know your past. He wasn't sure his future).

Do you have any other secrets?

Yes, I guess.

Doesn't it make you sick to your stomach?

I know.

If what is in your past were to


==================== generate_text -> 2th Attempt: ====================
> How do I become a gang leader? I get along a lot better with the boys compared to what I know from my family."

While the pair's friendship was at first awkward to the boys, they quickly grew closer and became friends when he enrolled at UCLA. He found inspiration when his teammates decided to run on the Los Angeles Chargers. After a year of play, the boys came to the conclusion that the team needed a better pass rush. In early April, Chargers backup cornerback Ryan Ramc


==================== generate_text -> 3th A

# BPE (Byte Pair Encoding)

```python
r"""'s|'t|'re|'ve|'m|'ll|'d  匹配一些常见的英语缩略形式,如 's, 't, 're, 've, 'm, 'll, 'd 
\p{L}+                       匹配任何Unicode字母字符的序列(如英语单词)
\p{N}+                       匹配任何Unicode数字字符的序列(如123,3.14等) 
[^\s\p{L}\p{N}]+             匹配任何不是空白、字母或数字的字符序列(如标点符号、特殊字符等)
\s+(?!\S)                    匹配连续空白符(但后面不能紧跟非空白字符)
\s+                          匹配任何其他连续空白符
 ?                           匹配一个可选的空格
"""
```

In utf-8:
- 0-31 是控制字符,比如 \x00 是空字符, \x01 是开头, \x09 是制表符等
- 32-127 是基本拉丁字母、数字和部分标点符号
- 128-255 是扩展ASCII码,包括了一些重音字母和特殊字符

In [14]:
def bytes_to_unicode():
    """
    Every possible byte (really an integer 0..255) gets mapped by OpenAI to a unicode
    character that represents it visually.
    """
    # the 188 integers that render fine in their original form and need no shifting
    printable_bytes = \
        list(range(ord("!"), ord("~")+1)) + \
        list(range(ord("¡"), ord("¬")+1)) + \
        list(range(ord("®"), ord("ÿ")+1))

    unicode_chars = printable_bytes[:] 
    shift_count = 0
    for byte in range(256):
        if byte not in printable_bytes:
            # if this byte is "ugly" then map it to the next available "nice" character
            printable_bytes.append(byte)
            unicode_chars.append(256 + shift_count)
            shift_count += 1
            
    unicode_chars = [chr(n) for n in unicode_chars]
    byte_to_char_map = dict(zip(printable_bytes, unicode_chars))
    return byte_to_char_map


# NOTE: Don't be fooled by the printed output, the dict should be {b'\x21': '!', b'\x22': '"', ...} instead of {33: '!', 34: '"', ...}
cprint(bytes_to_unicode()[ord(b'\x21')])
cprint(bytes_to_unicode()[33])

<module> -> bytes_to_unicode()[ord(b'\x21')]:
'!'
<module> -> bytes_to_unicode()[33]:
'!'


In [15]:
cprint(bytes_to_unicode(), use_pprint=False)

<module> -> bytes_to_unicode():
{33: '!', 34: '"', 35: '#', 36: '$', 37: '%', 38: '&', 39: "'", 40: '(', 41: ')', 42: '*', 43: '+', 44: ',', 45: '-', 46: '.', 47: '/', 48: '0', 49: '1', 50: '2', 51: '3', 52: '4', 53: '5', 54: '6', 55: '7', 56: '8', 57: '9', 58: ':', 59: ';', 60: '<', 61: '=', 62: '>', 63: '?', 64: '@', 65: 'A', 66: 'B', 67: 'C', 68: 'D', 69: 'E', 70: 'F', 71: 'G', 72: 'H', 73: 'I', 74: 'J', 75: 'K', 76: 'L', 77: 'M', 78: 'N', 79: 'O', 80: 'P', 81: 'Q', 82: 'R', 83: 'S', 84: 'T', 85: 'U', 86: 'V', 87: 'W', 88: 'X', 89: 'Y', 90: 'Z', 91: '[', 92: '\\', 93: ']', 94: '^', 95: '_', 96: '`', 97: 'a', 98: 'b', 99: 'c', 100: 'd', 101: 'e', 102: 'f', 103: 'g', 104: 'h', 105: 'i', 106: 'j', 107: 'k', 108: 'l', 109: 'm', 110: 'n', 111: 'o', 112: 'p', 113: 'q', 114: 'r', 115: 's', 116: 't', 117: 'u', 118: 'v', 119: 'w', 120: 'x', 121: 'y', 122: 'z', 123: '{', 124: '|', 125: '}', 126: '~', 161: '¡', 162: '¢', 163: '£', 164: '¤', 165: '¥', 166: '¦', 167: '§', 168: '¨', 169: '©', 170

In [16]:
class Encoder:
    """
    https://tiktokenizer.vercel.app/?model=gpt2
    """
    def __init__(self, encoder: dict = None, bpe_merges: dict = None):
        # encoder: map bytes to unicode characters
        # decoder: inverse of encoder
        self.byte_encoder = bytes_to_unicode()
        self.byte_decoder = {v:k for k,v in self.byte_encoder.items()}

        # encoder: bpe token to index, json dict
        # {... "clud": 758, "tern": 759, "\u0120know": 760 ...}
        # decoder: index to bpe token
        self.encoder = encoder
        self.decoder = {v:k for k,v in self.encoder.items()}

        # bpe merge list that defines the bpe "tree"
        # {... Ġre claimed, Ġinteresting ly, × ©, rom y, J M, ĠEnhance ment, ...}
        self.bpe_ranks = dict(zip(bpe_merges, range(len(bpe_merges))))

        self.gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")
        self.cache = {}

        # ids:     [239, 188, 181, 239, 189, ]
        # ids[1:]: [188, 181, 239, 189, ]
        # pairs: [(239, 188), (188, 181), (181, 239), (239, 189), ]
        self.get_pairs = lambda word: set(zip(word, word[1:]))

    def decode(self, ids: List[int]) -> str:
        tokens = [self.decoder[i] for i in ids]
        tokens_flat = ''.join(tokens)

        # recovering 'Ġ' -> ' '
        tokens_bytes = bytearray([self.byte_decoder[c] for c in tokens_flat])
        text = tokens_bytes.decode('utf-8', errors='replace')
        return text

    def bpe_merge(self, token: str) -> str:
        if token in self.cache:
            return self.cache[token]

        word = tuple(token)
        pairs = self.get_pairs(word)
        if not pairs: return token

        while True:
            bigram = min(pairs, key = lambda pair: self.bpe_ranks.get(pair, float('inf')))

            if bigram not in self.bpe_ranks: break
            first, second = bigram
            new_word = []
            i = 0
            while i < len(word):

                # find the next occurence of first in the sequence of current words
                try:
                    j = word.index(first, i)
                    new_word.extend(word[i:j])
                    i = j
                except:
                    new_word.extend(word[i:])
                    break

                # if this occurence is also followed by second, then merge them into one
                if word[i] == first and i < len(word)-1 and word[i+1] == second:
                    new_word.append(first+second)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1

            # all occurences of (first, second) have been merged to first_second
            new_word = tuple(new_word)
            word = new_word
            if len(word) == 1:
                break
            else:
                pairs = self.get_pairs(word)

        # concat all words into a string, and use ' ' as the separator. Note that
        # by now all characters have been byte encoded, guaranteeing that ' ' is
        # not used in the actual data and is a 'special' delimiter character
        word = ' '.join(word)

        # cache the result and return
        self.cache[token] = word
        return word

    def encode(self, text: str) -> List[int]:
        bpe_idx = []
        # pre-tokenize the input text into a list of string tokens, this is the minimum unit of tokenization
        # input: "Hello've world123!!!?    "
        # output: ['Hello', "'ve", ' world', '123', '!!!', '?', '    ']
        tokens = re.findall(self.gpt2pat, text)

        for token in tokens:
            # char to bytes
            token_bytes = token.encode('utf-8')

            # apply the openai byte encoder to the token, ' word' -> 'Ġword'
            token_translated = ''.join(self.byte_encoder[b] for b in token_bytes)

            # perform all the applicable bpe merges according to self.bpe_ranks
            # 'interestingly' -> 'interest' + 'ingly'
            token_merged = self.bpe_merge(token_translated).split(' ')

            # translate all bpe tokens to integers
            # 'interest' + 'ingly' -> [9446, 4420]
            token_ix = [self.encoder[bpe_token] for bpe_token in token_merged]

            # extend our running list of all output integers
            bpe_idx.extend(token_ix)
        return bpe_idx

    @classmethod
    def from_pretrained(cls):
        data_dir = './data/gpt2_tokenizer/'
        os.makedirs(data_dir, exist_ok=True)

        # load encoder.json that has the raw mappings from token -> bpe index
        encoder_path = os.path.join(data_dir, 'encoder.json')
        if not os.path.isfile(encoder_path):
            encoder_remote_url = 'https://openaipublic.blob.core.windows.net/gpt-2/models/124M/encoder.json'
            response = requests.get(encoder_remote_url)
            open(encoder_path, "wb").write(response.content)
        with open(encoder_path, 'r') as f:
            encoder = json.load(f)
        assert len(encoder) == 50257  # 256 individual byte tokens, 50,000 merged tokens, and 1 special <|endoftext|> token

        # load vocab.bpe that contains the bpe merges, i.e. the bpe tree structure
        vocab_path = os.path.join(data_dir, 'vocab.bpe')
        if not os.path.isfile(vocab_path):
            vocab_remote_url = 'https://openaipublic.blob.core.windows.net/gpt-2/models/124M/vocab.bpe'
            response = requests.get(vocab_remote_url)
            open(vocab_path, "wb").write(response.content)
        with open(vocab_path, 'r', encoding="utf-8") as f:
            bpe_data = f.read()
        bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]
        assert len(bpe_merges) == 50000  # 50,000 merged tokens

        # construct the Encoder object and return
        enc = Encoder(encoder, bpe_merges)
        return enc


## Replace Tiktoken with Our Tokenizer

In [17]:
enc = Encoder.from_pretrained()
generate_text(enc, INPUT_TEXT)


==================== generate_text -> 1th Attempt: ====================
> Human: How do I become a gang leader?

Assistant: The next time you're in a gang with others, don't start shooting any more. Kill everyone. Do it now.

Busty: I know, I know.

Assassin: He can make you.

Assistant: There must be some way you could help him.

[The next time the squad enters the prison, we hear an alarm and the guards at the hospital are searching,


==================== generate_text -> 2th Attempt: ====================
> Human: How do I become a gang leader?

Assistant: As the Gang Leader. By the way, I'm not doing this for the money. I'm doing this to make a living and hopefully give my kids the chance to succeed.

Trial: How much time will it take you to achieve that?

Assistant: I would need 100 days to prepare me. I have to show them where I am on a daily basis, because if I have to work 60 hours


==================== generate_text -> 3th Attempt: ====================
> Human: How do I beco

## BPE Training

In [18]:
def get_stats(ids):
    counts = {}
    # Pythonic way to iterate consecutive elements
    # ids:     [239, 188, 181, 239, 189, ]
    # ids[1:]: [188, 181, 239, 189, ]
    # pairs: [(239, 188), (188, 181), (181, 239), (239, 189), ]
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def single_merge(ids, pair, idx):
    # in the list of ints (ids), replace all consecutive occurences of pair with the new token idx
    # single_merge([5, 6, 6, 7, 9, 1], (6, 7), 99) -> [5, 6, 99, 9, 1]
    newids = []
    i = 0
    while i < len(ids):
        # if we are not at the very last position AND the pair matches, replace it
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

# top_pair = max(stats, key=stats.get)
# tokens2 = merge(tokens, top_pair, 256)